## Imports

In [1]:
# Imports for plots
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm #for better display of FITS images

# Imports from standard packages
# from scipy.interpolate import interp1d
from astropy.io import fits
import numpy as np

# Imports for extraction
from extract.overlap import TrpzOverlap, TrpzBox
from extract.throughput import ThroughputSOSS
from extract.convolution import WebbKer

# Local imports
from simulation_utils import load_simu, add_noise

### Matplotlib defaults

In [2]:
%matplotlib inline

In [3]:
plt.rc('figure', figsize=(9,3))
plt.rcParams["image.cmap"] = "inferno"

## Read ref files

In [4]:
# List of orders to consider in the extraction
order_list = [1, 2]

# Dict of ref files
PATH = '../jwst-mtl/SOSS/extract/'
REF_FILES = {'wave_maps': {1: 'Ref_files/wavelengths_m1.fits',
                           2: 'Ref_files/wavelengths_m2.fits'},
             'spat_pros': {1: 'Ref_files/spat_profile_m1.fits',
                           2: 'Ref_files/spat_profile_m2.fits'}}

#### Wavelength solution ####
wave_maps = [fits.getdata(PATH + REF_FILES['wave_maps'][order])
             for order in order_list]

#### Spatial profiles ####
spat_pros = [fits.getdata(PATH + REF_FILES['spat_pros'][order])
             for order in order_list]

# Convert data from fits files to float (fits precision is 1e-8)
wave_maps = [wv.astype('float64') for wv in wave_maps]
spat_pros = [p_ord.astype('float64') for p_ord in spat_pros]

#### Throughputs ####
thrpt_list = [ThroughputSOSS(order) for order in order_list]

#### Convolution kernels ####
ker_list = [WebbKer(wv_map) for wv_map in wave_maps]

# Put all inputs from reference files in a list
ref_files_args = [spat_pros, wave_maps, thrpt_list, ker_list]

## Load simulation

In [5]:
# Load a simulation
simu = load_simu("../Simulations/phoenix_teff_02300_scale_1.0e+02.fits")
scidata = simu["data"]

## Extraction

### Parameters
(Example using with few inputs parameters)

In [6]:
params = {}

# Map of expected noise (sig)
bkgd_noise = 20.

# Oversampling
params["n_os"] = 3

# Threshold on the spatial profile
params["thresh"] = 1e-4

### Init extraction object
(This can be done only once if the `n_os` doesn't change)

In [7]:
extra = TrpzOverlap(*ref_files_args, **params)

### Extract
Here, we run a really simple extraction (only one step, no tikhonov). <br>
In reality, this is were the "solver" should be used to iterate and get the best extraction as possible.

In [25]:
# Noise estimate to weight the pixels
# Poisson noise + background noise
sig = np.sqrt(scidata + bkgd_noise**2)

f_bin_list = []
for i in range(30):
    print(i)
    new_sig = sig.copy()
    # Generate a random mask
    new_mask = np.random.random(scidata.shape)
    new_sig[new_mask < 0.1] = np.inf
    # Extract
    f_k = extra.extract(data=scidata, sig=sig, mask=(new_mask < 0.1))
    
    f_bin_list.append(extra.bin_to_pixel(f_k=f_k, i_ord=0)[1])

0
1
2
3
4
5
6
7
8
9
10


KeyboardInterrupt: 

# Test b_n

In [8]:
sig = np.sqrt(scidata + bkgd_noise**2)
f_k = extra.extract(data=scidata, sig=sig)

In [24]:

%timeit extra.get_logl(f_k)

36.9 ms ± 823 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
